# Chains in LangChain

https://learn.deeplearning.ai/courses/langchain/lesson/4/chains

**Chains:** usually combine an LLM together with a prompt as a building block. You can put a bunch of these building blocks together to perform a sequence of operations on your text.  

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [3]:
%pip show langchain 

Name: langchain
Version: 0.2.10
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: f:\documentos\data_science\large language models llm\langchain_library_08_24\.venv\lib\site-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv, find_dotenv
import os
import warnings
from IPython.display import display, Markdown  # to see better the output text

warnings.filterwarnings('ignore')
_ = load_dotenv(find_dotenv())  # read local .env file

llm_model = "gpt-3.5-turbo"

In [5]:
import pandas as pd
df = pd.read_csv('resources/reviews.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [17]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(temperature=0., model=llm_model)


prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)
chain = LLMChain(llm=llm, prompt=prompt)
chain("bad")

{'adjective': 'bad',
 'text': "Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired."}

## LLMChain class is deprecated v0.1.17
    
Alternative we can use **RunnableSequence**, e.g., `prompt | llm`. Let's see an example

In [7]:

from langchain_core.output_parsers import StrOutputParser # to just get the text output

prompt =  PromptTemplate.from_template("Tell me a {adjective} joke")
chain = prompt | llm | StrOutputParser()
chain.invoke({"adjective":"bad"})

# equvalent to use
from langchain_core.runnables import RunnableSequence
sequence = RunnableSequence(first=prompt, middle=[llm], last=StrOutputParser())
sequence.invoke({"adjective":"bad"})

"Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired."

In [8]:
type(chain)

langchain_core.runnables.base.RunnableSequence

## SimpleSequentialChain --> RunnableSequence :
 The output os the one chain is the input of the next chain.
  
![simple-sequential-chain](resources/simple_sequential_chain.png)

In [9]:
from langchain.chains.sequential import SimpleSequentialChain

translate_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

description_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)

# deprecated
chain_one = LLMChain(llm=llm, prompt=translate_prompt)
chain_two = LLMChain(llm=llm, prompt=description_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                             )

overall_simple_chain.run( "Green Apples")

f:\DOCUMENTOS\DATA_SCIENCE\Large Language Models LLM\LangChain_library_08_24\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
"Green Apple Grove"
Green Apple Grove is an eco-friendly company specializing in organic produce and sustainable farming practices for a healthier planet.

> Finished chain.


'Green Apple Grove is an eco-friendly company specializing in organic produce and sustainable farming practices for a healthier planet.'

### We just get the output text

In [10]:
from langchain_core.runnables import RunnableSequence,Runnable

sequence_run = translate_prompt | {
    "company_name": llm} | description_prompt | {"llm_summary": llm}


type(sequence_run)

langchain_core.runnables.base.RunnableSequence

In [11]:
sequence_run.invoke({"product": "Green Apples"})

{'llm_summary': AIMessage(content='"Green Apple Grove" is a sustainable agriculture company specializing in organic farming practices and producing high-quality fruits and vegetables.', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 144, 'total_tokens': 167}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-07b34581-da37-479f-b4b8-097df9b4445a-0', usage_metadata={'input_tokens': 144, 'output_tokens': 23, 'total_tokens': 167})}

### To get the LLM response in each case

In [12]:
from operator import itemgetter

runnable1 = translate_prompt | llm | {"company_name": StrOutputParser()}

runnable2 = runnable1.assign(words_summary=itemgetter(
    "company_name") | description_prompt | llm | StrOutputParser())
type(runnable2)

langchain_core.runnables.base.RunnableSequence

In [13]:
runnable2.invoke({"product": "Green Apples"})

{'company_name': '"Green Apple Grove"',
 'words_summary': 'Green Apple Grove is a sustainable agriculture company specializing in organic apple farming, committed to environmental stewardship and community engagement.'}

## SequentialChain

There are many inputs and outputs, so we we use `RunnableSequence` toghether `RunnableAssign`(using *assign* function).

One important thing to note about  about all these subchains(runnables) is that the input and output keys have to be exactly the same each time they are used.

![sequential chain](resources/sequential_chain.png)

In [14]:
# prompt template 1: translate to english
from langchain_core.runnables import RunnableAssign, RunnableParallel
translate_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into {Language} language:"
    "\n\n{Review}"
)

# to save the input
ch = RunnableAssign(RunnableParallel())

# chain 1: input= Review and output= Translated_Review
chain_one = ch.assign(Translated_Review=translate_prompt | llm | StrOutputParser())

In [19]:
description_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{Translated_Review}"
)
# chain 2: input= Translated_Review and output= summary
chain_two = chain_one.assign(Summary= description_prompt | llm | StrOutputParser())

source_lang_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = chain_two.assign(Source_language = source_lang_prompt | llm | StrOutputParser())


In [22]:
final_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {Summary}\n\nLanguage: {Source_language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = chain_three.assign(Response=chain_three | final_prompt| llm | StrOutputParser())


In [23]:
chain_four.invoke({"Review": "I love this product, it is amazing!. So, I will buy it again. I just have to say that the delivery took three days longer than what was stated on the online sales checkout.", "Language": "español"})

{'Review': 'I love this product, it is amazing!. So, I will buy it again. I just have to say that the delivery took three days longer than what was stated on the online sales checkout.',
 'Language': 'español',
 'Translated_Review': '¡Me encanta este producto, es increíble! Así que lo compraré de nuevo. Solo tengo que decir que la entrega tardó tres días más de lo que se indicaba en el proceso de compra en línea.',
 'Summary': 'El producto es increíble y lo compraré de nuevo, aunque la entrega tardó tres días más de lo esperado.',
 'Source_language': 'English',
 'Response': 'Thank you for your feedback! We apologize for the delay in delivery, and we appreciate your understanding. We are glad to hear that you found the product amazing and that you would purchase it again. If you have any further feedback or questions, please feel free to reach out to us. Thank you for choosing our product!'}

## Router Chain
![alt](resources/router_chain.png)

In [27]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [28]:
# this information is going to be pass to the router chain, it can decide which chain to use
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [24]:
from langchain.prompts import PromptTemplate
from langchain.chains.router import MultiPromptChain  # wrapper for the router chain

# this use a LLM itself to decide which chain to use
from langchain.chains.router.llm_router import LLMRouterChain

# this pasrses the output of the LLMRouterChain to decide which chain to use
from langchain.chains.router.llm_router import RouterOutputParser

In [31]:

destination_chains = {} # dict to store the chains for each subject
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
destinations

['physics: Good for answering questions about physics',
 'math: Good for answering math questions',
 'History: Good for answering history questions',
 'computer science: Good for answering computer science questions']

In [38]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [32]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [35]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [40]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [41]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [42]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [47]:
# In this case the model should use the default chain but instead it detect the subject as biology and we don't have a chain for that
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'